# SHA-256 Password Cracker

I built a simple SHA-256 password "cracker" in Python to understand how password cracking works and the challenges behind breaking hashes.

## Step 1: Understanding SHA-256

Firstly I learnt about how the SHA-256 hashing algorithm works. I learnt that SHA-256 is a *one-way* function, meaning that it’s computationally infeasible to reverse a hash back into its original value. This is good for security, but it means I couldn’t just “decode” the hash directly. 

Instead, my goal was to compare hashes by re-hashing each password candidate in my wordlist and checking if it matched the target hash.

## Step 2: Setting Up the Environment

For this project, I didn’t need to install any extra libraries because Python’s built-in `hashlib` library includes SHA-256 hashing functionality. I created a small project folder, placed my `sha256_cracker.py` script inside, and added a `wordlist.txt` file containing a list of test passwords. I used a list of commonly used passwords for this test, but in a real-world scenario, attackers might use much larger wordlists or sophisticated techniques to generate password candidates.


## Step 3: Create a sha256_hash helper fundation

The helper function takes a string and returns its SHA-256 hash.  This will then be used in the main function to hash each commonly used password before comparision with the target hash.

In [ ]:
def sha256_hash(text):
    """
    Returns the SHA-256 hash of the given text (string).
    """
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

## Step 4: Create core password comparision function

The core logic is to compare the target hash against a list of passwords is implemented in the compare_againsts_most_popular_passwords_hashed() function. It reads the password file line by line, hashes each password, and compares the result to the target hash.

In [ ]:
# For testing purposes this function also breaks after 10 attempts.

def compare_againsts_most_popular_passwords_hashed(target_hash, password_file): 
    with open(password_file, "r", encoding='utf-8') as password_list:
        for attempt, password in enumerate(password_list):
            if attempt == 10:
                break
            print(f"[{attempt}] Attempting to crack: {target_hash}! \n")
            password = password.strip("\n")
            password_hash = sha256_hash(password)

            if password_hash == target_hash:
                return password
    return None

# Step 5: Testing and Learning

After implementing the cracking logic, I tested it by generating my own SHA-256 hash for a known password, then inserting it into the wordlist. The script successfully cracked the hash by matching it with the corresponding wordlist password. This was a good sanity check and confirmed that my approach was working.

Through this process, I learned that while SHA-256 is a strong hashing algorithm, it’s not perfect in every scenario. For example, unsalted hashes are vulnerable to brute-force and dictionary attacks, especially if the attacker has access to a wordlist of commonly used passwords.

# Reflections and lessons learned

Performance Considerations
Even in this basic scrit, I quickly ran into performance bottlenecks when trying to use larger wordlists as the script could take a while to process a huge file of potential passwords. In real-world scenarios, attackers would likely use more advanced tools, parallel processing, or even cloud services to speed up the process.

Security Implications
This exercise reinforced the importance of strong password policies, including:

Using complex passwords (e.g., mixing letters, numbers, and special characters).

Salting passwords before hashing. A salt is a random value added to the password before hashing, which makes it much harder for attackers to use precomputed hash tables (like rainbow tables) to crack passwords.

Overall, this project gave me a hands-on understanding of how password hashes work and how attackers might use simple brute-force or dictionary attacks to crack weak passwords. It also helped me realise why good hashing practices—like salting—are crucial for secure systems.

# Full Code

In [ ]:
from pwn import *
import sys
import hashlib

def sha256_hash(text):
    """
    Returns the SHA-256 hash of the given text (string).
    """
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

def compare_againsts_most_popular_passwords_hashed(target_hash, password_file): 
    with open(password_file, "r", encoding='utf-8') as password_list:
        for attempt, password in enumerate(password_list):
            if attempt == 10:
                break
            print(f"[{attempt}] Attempting to crack: {target_hash}! \n")
            password = password.strip("\n")
            password_hash = sha256_hash(password)

            if password_hash == target_hash:
                return password
    return None
        

if __name__ == "__main__":
    # Test Example
    target_hash = hashlib.sha256("test".encode('utf-8')).hexdigest()
    password_file = "1K-most-used-passwords-NCSC.txt"

    result = compare_againsts_most_popular_passwords_hashed(target_hash, password_file)
    if result is not None:
        print(f"[+] Password found: '{result}'")
    else:
        print("[!] No match found in the wordlist.")
